# Coin

A - монета неправильная

Bn - выпало n орлов
$$P(A|B_n) = \frac{P(A \cap B_n)}{P(B_n)} = \frac{P(B_n | A) * P(A)}{P(B_n | A) * P(A) + P(B_n | \neg A) * P(\neg A)} = \frac{0.9^{n} * 0.5}{0.9^{n} * 0.5 + 0.5^{n} * 0.5} \geqslant 0.95 $$

$$ 0.9^n \geqslant 0.95 * (0.9^n + 0.5^n) $$

$$ \frac{0.9^n}{20} \geqslant 0.95 * 0.5^n$$

$$ 1.8^n \geqslant 19 $$

$$ n \geqslant 6 $$

# PAM

$$ PAM^1 = 
\begin{bmatrix} 
1-3a & a & a & a \\
a & 1-3a & a & a \\
a & a & 1-3a & a \\
a & a & a & 1-3a  \\
\end{bmatrix}
$$

Тогда, исходя из симметрии, 

$$ PAM^k = 
\begin{bmatrix} 
1-3a_k & a_k & a_k & a_k \\
a_k & 1-3a_k & a_k & a_k \\
a_k & a_k & 1-3a_k & a_k \\
a_k & a_k & a_k & 1-3a_k  \\
\end{bmatrix}
$$

$$ PAM^{k + 1} = PAM^k * PAM ^ 1 \Rightarrow a_{k+1} = (1 - 3a) * a_k + a(1 - 3a_k) + aa_k + aa_k = a_k(1 - 4a) + a$$

$$ a_1 = a$$

$$ a_2 = a(1 - 4a) + a = 2a - 4a^2 = \frac{1}{4} - \frac{1}{4} + 2a - 4a^2 = \frac{1}{4}(1 - (1 - 8a +16a^2)) = \frac{1}{4}(1 - (1 - 4a)^2)$$

$$ a_3 = \frac{1}{4}(1 - (1 - 4a)^2) * (1 - 4a) + a = \frac{1}{4}(1 - 4a - (1 - 4a)^3 + 4a) = \frac{1}{4}(1 - (1 - 4a)^3) $$

Отсюда видим, что зависимость похожа на $$ a_n = \frac{1}{4}(1 - (1 - 4a)^n) $$ 
Докажем это с помощью мат индукции.
Пусть $$ a_n = \frac{1}{4}(1 - (1 - 4a)^n) $$
Тогда $$ a_{n + 1} = \frac{1}{4}(1 - (1 - 4a)^n) * (1 - 4a) + a = \frac{1}{4}(1 - 4a - (1 - 4a)^{n + 1} + 4a) = \frac{1}{4}(1 - (1 - 4a)^{n + 1}) $$


Значит
$$ a_n = \frac{1}{4}(1 - (1 - 4a)^n) $$

$$ 1 - 3a_n = 1 - \frac{3}{4}(1 - (1 - 4a)^n) = \frac{3(1 - 4a)^n + 1}{4} $$ 

$$ PAM^n = \frac{1}{4} * \begin{bmatrix} 
3(1 - 4a)^n + 1 & 1 - (1 - 4a)^n & 1 - (1 - 4a)^n & 1 - (1 - 4a)^n \\
1 - (1 - 4a)^n & 3(1 - 4a)^n + 1 & 1 - (1 - 4a)^n & 1 - (1 - 4a)^n \\
1 - (1 - 4a)^n & 1 - (1 - 4a)^n & 3(1 - 4a)^n + 1 & 1 - (1 - 4a)^n \\
1 - (1 - 4a)^n & 1 - (1 - 4a)^n & 1 - (1 - 4a)^n & 3(1 - 4a)^n + 1  \\
\end{bmatrix}
$$

# Viterbi + FB

In [17]:
import numpy as np
from collections import namedtuple

Probabilities = namedtuple('Probabilities', ('p', 'a11', 'a12', 'a21', 'a22', 'b1', 'b2'))

In [13]:
def viterbi(seq, t):
    n = len(seq)
    d = []
    l = t.p * t.b1[seq[0]]
    m = (1 - t.p) * t.b2[seq[0]]

    for i in range(1, n):
        sym = seq[i] 
        step1 = (l * t.a11 * t.b1[sym] < m * t.a21 * t.b1[sym])
        l = m * t.a21 * t.b1[sym] if step1 else l * t.a11 * t.b1[sym]
        step2 = (l * t.a12 * t.b2[sym] < m * t.a22 * t.b2[sym])
        m = m * t.a22 * t.b2[sym] if step2 else l * t.a12 * t.b2[sym] 
        d.append((step1, step2))

    step = l < m
    trail = []
    for i in range(n - 1):
        trail.append(int(step) + 1)
        step = d.pop()[step]
    return list(reversed(trail))
    

In [52]:
def fb(seq, t):
    n = len(seq)
    a, b = np.zeros((2, n)), np.zeros((2, n))
    a[0, 0] = t.p * t.b1[seq[0]]
    a[1, 0] = t.p * t.b2[seq[0]]
    b[0, n - 1] = 1
    b[1, n - 1] = 1
    for i in range(1, n):
        sym = seq[i]
        a1, a2 = a[0, i - 1], a[1, i - 1] 
        a[0, i] = a1 * t.a11 * t.b1[sym] + a2 * t.a21 * t.b1[sym]
        a[1, i] = a1 * t.a12 * t.b2[sym] + a2 * t.a22 * t.b2[sym]
        
        sym = seq[n - i]
        b1, b2 = b[0, n - i], b[1, n - i] 
        b[0, n - i - 1] = b1 * t.a11 * t.b1[sym] + b2 * t.a12 * t.b2[sym]
        b[1, n - i - 1] = b1 * t.a21 * t.b1[sym] + b2 * t.a22 * t.b2[sym]
    p1 = [round(a[0, i] * b[0, i] / (a[0, n - 1] + a[1, n - 1]), 3) for i in range(n)]
    p2 = [round(a[1, i] * b[1, i] / (a[0, n - 1] + a[1, n - 1]), 3) for i in range(n)]
    return p1, p2

## Test1

In [53]:
sequence = 'OPOPOPOOPPPPPPPPPPOOOOOOOO'

# choose first initial hidden state
p = 0.5
# change hidden state
a11 = 0.8
a12 = 0.2
a22 = 0.8
a21 = 0.2

# emit in first hidden state
b1 = {'O': 0.5, 'P': 0.5}

# emit in second hidden state
b2 = {'O': 0.1, 'P': 0.9}

prob = Probabilities(p, a11, a12, a21, a22, b1, b2)

In [55]:
res = viterbi(sequence, prob)
trace = fb(sequence, prob)
print("Viterbi:")
print(res)
print('FB:')
for v in trace:
    print(v)

Viterbi:
[1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1]
FB:
[0.86, 0.766, 0.878, 0.779, 0.887, 0.795, 0.911, 0.86, 0.457, 0.273, 0.19, 0.153, 0.139, 0.14, 0.154, 0.193, 0.279, 0.47, 0.889, 0.965, 0.978, 0.981, 0.981, 0.98, 0.974, 0.942]
[0.14, 0.234, 0.122, 0.221, 0.113, 0.205, 0.089, 0.14, 0.543, 0.727, 0.81, 0.847, 0.861, 0.86, 0.846, 0.807, 0.721, 0.53, 0.111, 0.035, 0.022, 0.019, 0.019, 0.02, 0.026, 0.058]


## Test2

In [56]:
sequence = 'OPOPOPOOPPPPPPPPPPOOOOOOOO'

# choose first initial hidden state
p = 0.5
# change hidden state
a11 = 0.5
a12 = 0.5
a22 = 0.5
a21 = 0.5

# emit in first hidden state
b1 = {'O': 0.5, 'P': 0.5}

# emit in second hidden state
b2 = {'O': 0.51, 'P': 0.49}

prob = Probabilities(p, a11, a12, a21, a22, b1, b2)

In [57]:
res = viterbi(sequence, prob)
trace = fb(sequence, prob)
print("Viterbi:")
print(res)
print('FB:')
for v in trace:
    print(v)

Viterbi:
[2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
FB:
[0.495, 0.505, 0.495, 0.505, 0.495, 0.505, 0.495, 0.495, 0.505, 0.505, 0.505, 0.505, 0.505, 0.505, 0.505, 0.505, 0.505, 0.505, 0.495, 0.495, 0.495, 0.495, 0.495, 0.495, 0.495, 0.495]
[0.505, 0.495, 0.505, 0.495, 0.505, 0.495, 0.505, 0.505, 0.495, 0.495, 0.495, 0.495, 0.495, 0.495, 0.495, 0.495, 0.495, 0.495, 0.505, 0.505, 0.505, 0.505, 0.505, 0.505, 0.505, 0.505]
